# Tabular data handling

This module defines the main class to handle tabular data in the fastai library: [`TabularDataset`](/tabular.data.html#TabularDataset). As always, there is also a helper function to quickly get your data.

To allow you to easily create a [`Learner`](/basic_train.html#Learner) for your data, it provides [`get_tabular_learner`](/tabular.data.html#get_tabular_learner).

In [1]:
from fastai.gen_doc.nbdoc import *
from fastai.tabular import * 
from fastai import *

In [2]:
show_doc(TabularDataBunch, doc_string=False)

<h2 id="TabularDataBunch"><code>class</code> <code>TabularDataBunch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L70" class="source_link">[source]</a></h2>

> <code>TabularDataBunch</code>(`train_dl`:[`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), `valid_dl`:[`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), `test_dl`:`Optional`\[[`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)\]=`None`, `device`:[`device`](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device)=`None`, `tfms`:`Optional`\[`Collection`\[`Callable`\]\]=`None`, `path`:`PathOrStr`=`'.'`, `collate_fn`:`Callable`=`'data_collate'`) :: [`DataBunch`](/basic_data.html#DataBunch)

The best way to quickly get your data in a [`DataBunch`](/basic_data.html#DataBunch) suitable for tabular data is to organize it in two (or three) dataframes. One for training, one for validation, and if you have it, one for testing. Here we are interested in a subsample of the [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult).

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
train_df, valid_df = df[:800].copy(),df[800:].copy()
train_df.head()

age          workclass  fnlwgt     education  education-num  \
0   49            Private  101320    Assoc-acdm           12.0   
1   44            Private  236746       Masters           14.0   
2   38            Private   96185       HS-grad            NaN   
3   38       Self-emp-inc  112847   Prof-school           15.0   
4   42   Self-emp-not-inc   82297       7th-8th            NaN   

        marital-status        occupation    relationship                 race  \
0   Married-civ-spouse               NaN            Wife                White   
1             Divorced   Exec-managerial   Not-in-family                White   
2             Divorced               NaN       Unmarried                Black   
3   Married-civ-spouse    Prof-specialty         Husband   Asian-Pac-Islander   
4   Married-civ-spouse     Other-service            Wife                Black   

       sex  capital-gain  capital-loss  hours-per-week  native-country  >=50k  
0   Female             0          1902              40   United-States      1  
1     Male         10520             0              45   United-States      1  
2   Female             0             0              32   United-States      0  
3     Male             0             0              40   United-States      1  
4   Female             0             0              50   United-States      0

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
dep_var = '>=50k'

In [3]:
show_doc(TabularDataBunch.from_df, doc_string=False)

<h4 id="TabularDataBunch.from_df"><code>from_df</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L72" class="source_link">[source]</a></h4>

> <code>from_df</code>(`path`, `train_df`:`DataFrame`, `valid_df`:`DataFrame`, `dep_var`:`str`, `test_df`:`OptDataFrame`=`None`, `tfms`:`Optional`\[`Collection`\[[`TabularTransform`](/tabular.transform.html#TabularTransform)\]\]=`None`, `cat_names`:`OptStrList`=`None`, `cont_names`:`OptStrList`=`None`, `stats`:`OptStats`=`None`, `log_output`:`bool`=`False`, `kwargs`) → [`DataBunch`](/basic_data.html#DataBunch)

Creates a [`DataBunch`](/basic_data.html#DataBunch) in `path` from `train_df`, `valid_df` and optionally `test_df`. The dependent variable is `dep_var`, while the categorical and continuous variables are in the `cat_names` columns and `cont_names` columns respectively. If `cont_names` is None then we assume all variables that aren't dependent or categorical are continuous. The [`TabularTransform`](/tabular.transform.html#TabularTransform) in `tfms` are applied to the dataframes as preprocessing, then the categories are replaced by their codes+1 (leaving 0 for `nan`) and the continuous variables are normalized. You can pass the `stats` to use for that step. If `log_output` is True, the dependant variable is replaced by its log.

Note that the transforms should be passed as `Callable`: the actual initialization with `cat_names` and `cont_names` is done inside.

In [ ]:
tfms = [FillMissing, Categorify]
data = TabularDataBunch.from_df(path, train_df, valid_df, dep_var=dep_var, tfms=tfms, cat_names=cat_names)

 You can then easily create a [`Learner`](/basic_train.html#Learner) for this data with [`get_tabular_learner`](/tabular.data.html#get_tabular_learner).

In [4]:
show_doc(get_tabular_learner)

<h4 id="get_tabular_learner"><code>get_tabular_learner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L89" class="source_link">[source]</a></h4>

> <code>get_tabular_learner</code>(`data`:[`DataBunch`](/basic_data.html#DataBunch), `layers`:`Collection`\[`int`\], `emb_szs`:`Dict`\[`str`, `int`\]=`None`, `metrics`=`None`, `ps`:`Collection`\[`float`\]=`None`, `emb_drop`:`float`=`0.0`, `y_range`:`OptRange`=`None`, `use_bn`:`bool`=`True`, `kwargs`)

Get a [`Learner`](/basic_train.html#Learner) using `data`, with `metrics`, including a [`TabularModel`](/tabular.models.html#TabularModel) created using the remaining params.  

`emb_szs` is a `dict` mapping categorical column names to embedding sizes; you only need to pass sizes for columns where you want to override the default behaviour of the model.

In [5]:
show_doc(TabularDataset, doc_string=False)

<h2 id="TabularDataset"><code>class</code> <code>TabularDataset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L20" class="source_link">[source]</a></h2>

> <code>TabularDataset</code>(`df`:`DataFrame`, `dep_var`:`str`, `cat_names`:`OptStrList`=`None`, `cont_names`:`OptStrList`=`None`, `stats`:`OptStats`=`None`, `log_output`:`bool`=`False`) :: [`DatasetBase`](/basic_data.html#DatasetBase)

A dataset from `DataFrame` `df` with the dependent being the `dep_var` column, while the categorical and continuous variables are in the `cat_names` columns and `cont_names` columns respectively. Categories are replaced by their codes+1 (leaving 0 for `nan`) and the continuous variables are normalized. You can pass the `stats` to use for normalization; if none, then will be calculated from your data. If the flag `log_output` is True, the dependant variable is replaced by its log.

In [6]:
show_doc(TabularDataset.from_dataframe, doc_string=False)

<h4 id="TabularDataset.from_dataframe"><code>from_dataframe</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L52" class="source_link">[source]</a></h4>

> <code>from_dataframe</code>(`df`:`DataFrame`, `dep_var`:`str`, `tfms`:`Optional`\[`Collection`\[[`TabularTransform`](/tabular.transform.html#TabularTransform)\]\]=`None`, `cat_names`:`OptStrList`=`None`, `cont_names`:`OptStrList`=`None`, `stats`:`OptStats`=`None`, `log_output`:`bool`=`False`) → `TabularDataset`

Factory method to create a [`TabularDataset`](/tabular.data.html#TabularDataset) from `df`. The only difference from the constructor is that it gets a list `tfms` of `TabularTfm` that it applied before passing the dataframe to the class initialization.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [7]:
show_doc(TabularDataset.get_emb_szs)

<h4 id="TabularDataset.get_emb_szs"><code>get_emb_szs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/data.py#L50" class="source_link">[source]</a></h4>

> <code>get_emb_szs</code>(`sz_dict`)